# 🐼 Pandas Review for ML/DL

**Mục tiêu:** Ôn tập Pandas operations cho data preprocessing

**Nội dung:**
- DataFrame creation & inspection
- Indexing & selection
- Data cleaning (missing values, duplicates)
- Transformation & feature engineering
- GroupBy operations
- Merge & join
- Time series basics

**Level:** Intermediate

---

In [ ]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 20)
pd.set_option('display.precision', 3)

print(f"Pandas version: {pd.__version__}")

---

## 1. DataFrame Creation & Inspection

### Common patterns in ML workflows

In [ ]:
# Create sample dataset (typical ML scenario)
np.random.seed(42)
n_samples = 1000

data = {
    'user_id': np.arange(n_samples),
    'age': np.random.randint(18, 70, n_samples),
    'income': np.random.lognormal(10, 1, n_samples),
    'clicks': np.random.poisson(5, n_samples),
    'conversions': np.random.binomial(1, 0.1, n_samples),
    'category': np.random.choice(['A', 'B', 'C'], n_samples),
    'signup_date': pd.date_range('2023-01-01', periods=n_samples, freq='h')
}

df = pd.DataFrame(data)

# Add some missing values (realistic)
df.loc[np.random.choice(df.index, 50, replace=False), 'income'] = np.nan
df.loc[np.random.choice(df.index, 30, replace=False), 'clicks'] = np.nan

# Quick inspection
print("Shape:", df.shape)
print("\nData types:")
print(df.dtypes)
print("\nFirst 3 rows:")
print(df.head(3))
print("\nBasic stats:")
print(df.describe())
print("\nMissing values:")
print(df.isnull().sum())

### 💡 Essential Inspection Methods

```python
df.info()              # Overview: dtypes, memory, non-null counts
df.describe()          # Statistical summary
df.shape               # (rows, columns)
df.columns             # Column names
df.dtypes              # Column data types
df.isnull().sum()      # Missing value counts
df.nunique()           # Number of unique values per column
df.memory_usage(deep=True)  # Memory footprint
```

## 2. Indexing & Selection

### loc vs iloc vs [] (know the difference!)

In [ ]:
# [] - Column selection or boolean indexing
ages = df['age']                          # Single column (Series)
subset = df[['age', 'income']]            # Multiple columns (DataFrame)
high_income = df[df['income'] > 50000]    # Boolean indexing

# .loc[] - Label-based indexing
row_0 = df.loc[0]                         # Single row by label
rows_0_to_5 = df.loc[0:5]                 # Rows 0 to 5 (INCLUSIVE)
specific = df.loc[df['age'] > 50, ['age', 'income']]  # Boolean rows, specific columns

# .iloc[] - Position-based indexing
first_row = df.iloc[0]                    # First row
first_5_rows = df.iloc[:5]                # First 5 rows (EXCLUSIVE)
slice_2d = df.iloc[:10, :3]               # First 10 rows, first 3 columns

# Query (SQL-like, convenient)
filtered = df.query('age > 50 and income > 30000')
filtered2 = df.query('category == "A" and clicks > @df.clicks.median()')  # @ for variables

print(f"High income users: {len(high_income)}")
print(f"Filtered (query): {len(filtered)}")
print(f"\nSample:")
print(df.loc[0:2, ['age', 'income', 'category']])

### ⚠️ Chained Assignment Warning

```python
# ❌ BAD: Chained assignment (may not work)
df[df['age'] > 50]['income'] = 0  # Warning!

# ✅ GOOD: Use .loc
df.loc[df['age'] > 50, 'income'] = 0
```

## 3. Data Cleaning

### Missing Values

In [ ]:
# Identify missing values
missing_summary = df.isnull().sum()
missing_pct = (df.isnull().sum() / len(df)) * 100

print("Missing values:")
print(pd.DataFrame({'Count': missing_summary, 'Percentage': missing_pct}))

# Strategy 1: Drop rows with any missing
df_dropped = df.dropna()  # Drops rows with ANY missing value

# Strategy 2: Drop rows with missing in specific columns
df_dropped2 = df.dropna(subset=['income', 'clicks'])

# Strategy 3: Fill with constant
df_filled = df.fillna({'income': 0, 'clicks': 0})

# Strategy 4: Fill with statistics
df_filled2 = df.copy()
df_filled2['income'] = df_filled2['income'].fillna(df_filled2['income'].median())
df_filled2['clicks'] = df_filled2['clicks'].fillna(df_filled2['clicks'].mean())

# Strategy 5: Forward/backward fill (time series)
df_filled3 = df.fillna(method='ffill')  # Forward fill

print(f"\nOriginal: {len(df)} rows")
print(f"After dropna(): {len(df_dropped)} rows")
print(f"After fillna (median): {df_filled2['income'].isnull().sum()} missing")

### Duplicates & Data Quality

In [ ]:
# Check duplicates
print(f"Duplicates: {df.duplicated().sum()}")

# Check duplicates on specific columns
print(f"Duplicate user_ids: {df.duplicated(subset=['user_id']).sum()}")

# Remove duplicates (keep first occurrence)
df_dedup = df.drop_duplicates(subset=['user_id'], keep='first')

# Data validation
print("\nData quality checks:")
print(f"  Negative age: {(df['age'] < 0).sum()}")
print(f"  Negative income: {(df['income'] < 0).sum()}")
print(f"  Invalid category: {(~df['category'].isin(['A', 'B', 'C'])).sum()}")

# Remove outliers (simple method)
def remove_outliers(df, column, n_std=3):
    mean = df[column].mean()
    std = df[column].std()
    return df[(df[column] >= mean - n_std*std) & (df[column] <= mean + n_std*std)]

df_no_outliers = remove_outliers(df, 'income', n_std=3)
print(f"\nAfter outlier removal: {len(df)} -> {len(df_no_outliers)} rows")

## 4. Feature Engineering

### Create new features

In [ ]:
# Numeric transformations
df['log_income'] = np.log1p(df['income'])  # log1p(x) = log(1+x) (handles 0)
df['income_per_age'] = df['income'] / df['age']
df['ctr'] = df['conversions'] / (df['clicks'] + 1)  # Click-through rate

# Binning (discretization)
df['age_group'] = pd.cut(df['age'], bins=[0, 30, 50, 100], labels=['Young', 'Middle', 'Senior'])
df['income_quartile'] = pd.qcut(df['income'], q=4, labels=['Q1', 'Q2', 'Q3', 'Q4'])

# Datetime features
df['hour'] = df['signup_date'].dt.hour
df['day_of_week'] = df['signup_date'].dt.dayofweek
df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)

# Categorical encoding
# One-hot encoding
category_dummies = pd.get_dummies(df['category'], prefix='cat')
df = pd.concat([df, category_dummies], axis=1)

# Label encoding (simple)
df['category_encoded'] = df['category'].astype('category').cat.codes

# Interaction features
df['age_x_clicks'] = df['age'] * df['clicks']

print("New features created:")
print(df[['age', 'age_group', 'income', 'log_income', 'hour', 'is_weekend']].head())
print(f"\nShape after feature engineering: {df.shape}")

### 💡 Common Feature Engineering Patterns

```python
# Aggregation features (group statistics)
df['user_avg_clicks'] = df.groupby('user_id')['clicks'].transform('mean')

# Lag features (time series)
df['prev_clicks'] = df['clicks'].shift(1)

# Rolling statistics
df['rolling_mean_7d'] = df['clicks'].rolling(window=7).mean()

# Target encoding (be careful of leakage!)
target_means = df.groupby('category')['conversions'].mean()
df['category_target_enc'] = df['category'].map(target_means)
```

## 5. GroupBy Operations

### Split-Apply-Combine

In [ ]:
# Basic groupby
category_stats = df.groupby('category').agg({
    'age': ['mean', 'std'],
    'income': ['mean', 'median'],
    'clicks': 'sum',
    'conversions': 'sum'
})

print("Category statistics:")
print(category_stats)

# Multiple groupby keys
group_stats = df.groupby(['category', 'age_group'])['income'].agg(['mean', 'count'])
print("\nCategory x Age Group:")
print(group_stats.head(6))

# Transform (broadcast aggregation back)
df['category_mean_income'] = df.groupby('category')['income'].transform('mean')
df['income_vs_category_mean'] = df['income'] - df['category_mean_income']

# Apply custom function
def custom_metric(group):
    return (group['conversions'].sum() / group['clicks'].sum()) if group['clicks'].sum() > 0 else 0

category_ctr = df.groupby('category').apply(custom_metric)
print("\nCategory CTR:")
print(category_ctr)

# Filter groups
large_groups = df.groupby('category').filter(lambda x: len(x) > 300)
print(f"\nLarge groups: {len(large_groups)} rows")

### Named Aggregations (Pandas 0.25+)

Cleaner syntax:

In [ ]:
# Named aggregations (better column names)
summary = df.groupby('category').agg(
    avg_age=('age', 'mean'),
    median_income=('income', 'median'),
    total_clicks=('clicks', 'sum'),
    conversion_rate=('conversions', 'mean'),
    count=('user_id', 'count')
)

print("Summary with named aggregations:")
print(summary)

## 6. Merge & Join

### Combining DataFrames

In [ ]:
# Create additional data
user_metadata = pd.DataFrame({
    'user_id': np.random.choice(df['user_id'], 500),
    'country': np.random.choice(['US', 'UK', 'DE', 'FR'], 500),
    'premium': np.random.choice([0, 1], 500)
})

# Inner join (only matching keys)
merged_inner = df.merge(user_metadata, on='user_id', how='inner')

# Left join (keep all from left)
merged_left = df.merge(user_metadata, on='user_id', how='left')

# Multiple key join
# merged = df1.merge(df2, on=['key1', 'key2'], how='inner')

# Join on index
# merged = df1.join(df2, how='left')

print(f"Original df: {len(df)} rows")
print(f"User metadata: {len(user_metadata)} rows")
print(f"Inner merge: {len(merged_inner)} rows")
print(f"Left merge: {len(merged_left)} rows")
print(f"Missing after left join: {merged_left['country'].isnull().sum()}")

# Concatenate (stack DataFrames)
df1 = df.head(100)
df2 = df.tail(100)
stacked = pd.concat([df1, df2], axis=0, ignore_index=True)
print(f"\nStacked: {len(stacked)} rows")

### 💡 Merge Types

```python
# how='inner': Only matching keys (intersection)
# how='left': All from left, matching from right
# how='right': All from right, matching from left
# how='outer': All keys (union)

# Indicator to track merge source
merged = df1.merge(df2, on='key', how='outer', indicator=True)
merged['_merge'].value_counts()
# left_only: Only in df1
# right_only: Only in df2
# both: In both
```

## 7. Advanced Operations

### Apply, Map, Applymap

In [ ]:
# apply: Apply function along axis
# Row-wise (axis=1)
def compute_score(row):
    return row['age'] * 0.1 + row['clicks'] * 2

df['score'] = df.apply(compute_score, axis=1)

# Column-wise (axis=0)
numeric_cols = ['age', 'income', 'clicks']
normalized = df[numeric_cols].apply(lambda x: (x - x.mean()) / x.std(), axis=0)

# map: Element-wise for Series
category_map = {'A': 'Type_A', 'B': 'Type_B', 'C': 'Type_C'}
df['category_mapped'] = df['category'].map(category_map)

# applymap: Element-wise for DataFrame (deprecated, use map)
# df_rounded = df[numeric_cols].applymap(lambda x: round(x, 2))

print("Score sample:")
print(df[['age', 'clicks', 'score']].head())
print("\nNormalized sample:")
print(normalized.head())

### Vectorized String Operations

In [ ]:
# Create text column
df['user_name'] = ['User_' + str(i) for i in range(len(df))]

# String operations (vectorized)
df['name_length'] = df['user_name'].str.len()
df['name_upper'] = df['user_name'].str.upper()
df['name_contains_5'] = df['user_name'].str.contains('5')

# Extract patterns
df['user_num'] = df['user_name'].str.extract(r'(\d+)').astype(int)

print("String operations:")
print(df[['user_name', 'name_length', 'name_contains_5', 'user_num']].head())

## 8. Performance Tips

### Optimize your Pandas code

In [ ]:
import time

# Create large DataFrame
large_df = pd.DataFrame({
    'A': np.random.randn(100000),
    'B': np.random.randn(100000),
    'C': np.random.choice(['X', 'Y', 'Z'], 100000)
})

# ❌ SLOW: Iterrows
start = time.time()
result = []
for idx, row in large_df.head(1000).iterrows():
    result.append(row['A'] + row['B'])
time_iterrows = time.time() - start

# ✅ FAST: Vectorized
start = time.time()
result_vec = large_df.head(1000)['A'] + large_df.head(1000)['B']
time_vectorized = time.time() - start

print(f"Iterrows: {time_iterrows:.4f}s")
print(f"Vectorized: {time_vectorized:.4f}s")
print(f"Speedup: {time_iterrows/time_vectorized:.1f}x")

# Other tips
print("\n💡 Performance tips:")
print("1. Use vectorized operations (avoid iterrows)")
print("2. Use query() instead of boolean indexing for large datasets")
print("3. Use categorical dtype for low-cardinality strings")
print("4. Read CSV with dtype specification and usecols")
print("5. Use eval() for complex expressions")

# Categorical dtype example
df['category'] = df['category'].astype('category')
print(f"\nMemory saved with categorical: ~{df.memory_usage()['category'] / 1024:.1f} KB")

## 9. ML Pipeline Integration

### Convert to NumPy/ML-ready format

In [ ]:
# Prepare for ML
feature_cols = ['age', 'log_income', 'clicks', 'hour', 'is_weekend', 'cat_A', 'cat_B', 'cat_C']
target_col = 'conversions'

# Handle missing values
df_ml = df[feature_cols + [target_col]].copy()
df_ml = df_ml.dropna()

# Convert to NumPy
X = df_ml[feature_cols].values
y = df_ml[target_col].values

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"X dtype: {X.dtype}")
print(f"Class balance: {y.mean():.2%} positive")

# Save/load
# df.to_csv('data.csv', index=False)
# df.to_parquet('data.parquet')  # Faster, smaller
# df = pd.read_parquet('data.parquet')

---

## 🎯 Key Takeaways

### Must Know for ML/DL

1. **Indexing**: `.loc[]` (label), `.iloc[]` (position), `[]` (columns/boolean)
2. **Missing values**: `dropna()`, `fillna()`, check with `isnull()`
3. **GroupBy**: Essential for feature engineering and EDA
4. **Merge**: Combine datasets (`inner`, `left`, `right`, `outer`)
5. **Vectorization**: ALWAYS prefer over loops
6. **Datetime**: Extract features with `.dt` accessor

### Common ML Preprocessing

```python
# 1. Load data
df = pd.read_csv('data.csv')

# 2. Handle missing
df = df.dropna(subset=['important_col'])
df['other_col'] = df['other_col'].fillna(df['other_col'].median())

# 3. Feature engineering
df['new_feature'] = df['col1'] / df['col2']
df['category_encoded'] = df['category'].astype('category').cat.codes

# 4. Split features and target
X = df[feature_cols].values
y = df[target_col].values
```

### Performance

1. Vectorize everything
2. Use `query()` for complex filters
3. Categorical dtype for string columns
4. Specify dtypes when reading CSV
5. Use Parquet for large datasets

---

**Next:** Matplotlib & Seaborn for visualization